<a href="https://colab.research.google.com/github/SELF-msselve/UTN/blob/main/CEL_Real_time_Ingesta_de_eventos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción
Cuando se trata de datos en tiempo real, las fuentes u orígenes transmiten datos, en intervalos regulares o irregulares. Esos datos datos en tiempo real, podríamos llamarlos eventos o mensajes.

Los mensajes deben ser enviados o dirigidos a una plataforma especial, como Apache Kafka. Otras alternativas a Kafka son Apache Pulsar, Amazon Kinesis, Google PubSub, Azure Event Hubs, entre otros.

Estas plataformas facilitan y estandarizan el acceso y la disponibilización a este tipo de datos. Sin estas plataformas, el intercambio de datos entre varios sistemas sería costoso y difícil de mantener, como se refleja en esta [imágen](https://www.conduktor.io/kafka/_next/image/?url=https%3A%2F%2Fimages.ctfassets.net%2Fo12xgu4mepom%2F15MPdLa9Vh84mmRt8stbU0%2Fb6c1edfc8d23b88e63a06f89c2b2e1af%2FWhat_is_Apache_Kafka_Part_1_-_Data_Integration_Challenges.png&w=1920&q=75). A diferencia de esta [imágen](https://www.conduktor.io/kafka/_next/image/?url=https%3A%2F%2Fimages.ctfassets.net%2Fo12xgu4mepom%2FZm4Nu6YFkYuZrBoALR2yw%2F4f2063a4c2d070d0cf0a9f56b1624e64%2FWhat_is_Apache_Kafka_Part_1_-_Use_Cases_and_Applications.png&w=3840&q=75) donde los datos se concentran en Kafka.

Por último, [acá](https://eng.uber.com/wp-content/uploads/2016/08/image00.png) podes ver, en alto nivel, como Uber usa Kafka para entregar diferentes tipos de eventos a diferentes sistemas (Apps mobiles, modelos de data sciences, reportes, etc.)

### Instalación de servicios
Descargamos e instalamos los requisitos.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -c https://dlcdn.apache.org/kafka/3.5.0/kafka_2.13-3.5.0.tgz

--2023-08-13 15:27:20--  https://dlcdn.apache.org/kafka/3.5.0/kafka_2.13-3.5.0.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106792776 (102M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.5.0.tgz’

kafka_2.13-3.5.0.tg 100%[===================>] 101.84M   289MB/s    in 0.4s    

2023-08-13 15:27:21 (289 MB/s) - ‘kafka_2.13-3.5.0.tgz’ saved [106792776/106792776]



Descomprimimos el paquete de Kafka y renombrar la carpeta extraída

In [ ]:
!tar -xzf kafka_2.13-3.5.0.tgz
!mv kafka_2.13-3.5.0/ kafka

### Inicio de servicios
Ejecutamos unos comandos para iniciar el servidor de Kafka

In [ ]:
!./kafka/bin/zookeeper-server-start.sh -daemon ./kafka/config/zookeeper.properties
!./kafka/bin/kafka-server-start.sh -daemon ./kafka/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


## Creación de un tópico
[Animación](https://miro.medium.com/v2/resize:fit:1200/1*QWzbTNfsPNuEK69A_dtxCA.gif) de como funciona un tópico

[Animación](https://miro.medium.com/v2/resize:fit:1200/1*5EJ12zSrJCiY2BFdvGFNzg.gif) de como funciona un tópico y sus particiones

In [ ]:
!./kafka/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --partitions 3 --replication-factor 1 --topic weather_station

Created topic weather_station.


## Socket

A continuación vamos a consumir datos de una estación meteorológica, gestionado por el proyecto [HPWREN](https://hpwren.ucsd.edu/) de la Universidad California San Diego. Los datos son accesibles en tiempo real gracias a su infraestructurada de red conectada a internet.

El acceso a los datos será por medio de socket.

Un socket es una abstracción de programación que permite que los programas se comuniquen entre sí, mediante **el envío y recepción de datos a través de la red**, ya sea en el mismo equipo o en diferentes máquinas en una red local o en Internet.
En términos simples, **un socket es un punto final para enviar o recibir datos a través de una red, o bien es como una puerta que se abre en una máquina para permitir que los datos entren o salgan.**

El siguiente script utiliza la librería `socket` para crear y manejar sockets. La librería proporciona funciones y clases que permiten la creación, configuración y uso de sockets de red. En este caso específico, el script crea un socket usando el protocolo IPv4 (AF_INET) y el tipo de socket de flujo (SOCK_STREAM), que se utiliza para la comunicación fiable y orientada a conexión, como en el caso de las conexiones TCP.

El script establece una conexión con un host remoto y un puerto específico usando el método `connect()`. Una vez que la conexión se establece, el bucle principal recibe y procesa los datos que llegan al socket, y finalmente se cierra el socket al finalizar la ejecución del script.

In [ ]:
import socket

# Crear un socket TCP/IP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conectar el socket al host y puerto proporcionados
s.connect(('rtd.hpwren.ucsd.edu', 12020))

try:
  # Bucle principal para recibir y procesar datos en tiempo real
  while True:
      # Recibir datos del socket en bloques de hasta 1024 bytes
      data = s.recv(1024)

      # Decodificar los datos binarios en un string legible
      data = data.decode()

      # Imprimir los datos en la consola (puedes comentar esta línea si no es necesario)
      print(data)

except KeyboardInterrupt:
      # Manejar la interrupción del teclado (Ctrl+C)
      print("Deteniendo el script...")

except Exception as e:
      # Capturar y mostrar otros tipos de excepciones que puedan ocurrir
      print("Ocurrió un error")
      print(f"Error: {e}")

finally:
      # Cerrar el socket al finalizar la ejecución del script
      s.close()

Para saber que significa cada variable como Sn, Sm, Sx, Ta, Pa, etc. podes consultar este [link ](https://github.com/words-sdsc/coursera/blob/master/big-data-2/sensor/wxt-520-format.txt)

## Kafka producer

In [ ]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
from kafka import KafkaProducer

bootstrap_servers = 'localhost:9092'  # Dirección y puerto del servidor Kafka
topic = 'weather_station'  # Nombre del tópico en el que se enviarán los datos

producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

In [ ]:
import socket

# Crear un socket TCP/IP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conectar el socket al host y puerto proporcionados
s.connect(('rtd.hpwren.ucsd.edu', 12020))

try:
  # Bucle principal para recibir y procesar datos en tiempo real
  while True:
      # Recibir datos del socket en bloques de hasta 1024 bytes
      data = s.recv(1024)

      # Decodificar los datos binarios en un string legible
      data = data.decode()

      # Enviar los datos a un tópico
      producer.send(topic, data.encode("utf-8"))
      print(f"Se ha enviado el mensaje: {data}")

except KeyboardInterrupt:
      # Manejar la interrupción del teclado (Ctrl+C)
      print("Deteniendo el script...")

except Exception as e:
      # Capturar y mostrar otros tipos de excepciones que puedan ocurrir
      print("Ocurrió un error")
      print(f"Error: {e}")

finally:
      # Cerrar el socket al finalizar la ejecución del script
      s.close()

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940879	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940880	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940881	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940882	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R2:4:0	1691940882	0R2,Ta=24.5C,Ua=44.2P,Pa=886.9H

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940883	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940884	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

Se ha enviado el mensaje: 198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940885	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#

## Consumer

### Consola

In [ ]:
!./kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic weather_station --from-beginning

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940879	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940880	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940881	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940882	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R2:4:0	1691940882	0R2,Ta=24.5C,Ua=44.2P,Pa=886.9H

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940883	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940884	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940885	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940886	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#,Sm=99.9#,Sx=0.0#

198.202.124.3	HPWREN:LP-WXT536:0R1:4:0	1691940887	0R1,Dn=000#,Dm=000#,Dx=000#,Sn=0.0#

## Python

In [ ]:
from kafka import KafkaConsumer
from datetime import datetime

# To consume latest messages and auto-commit offsets
consumer = KafkaConsumer('weather_station',
                         bootstrap_servers=['localhost:9092'],
                         auto_offset_reset='earliest',
                         enable_auto_commit=False)

for message in consumer:
  data = message.value.decode()
  ip, id, tstamp, values = data.split("\t")
  for val in values.split(",")[1:]:
    variable, measurement = val.strip().split("=")
    print({variable: measurement, "tstamp": tstamp})

{'Dn': '000#', 'tstamp': '1691940879'}
{'Dm': '000#', 'tstamp': '1691940879'}
{'Dx': '000#', 'tstamp': '1691940879'}
{'Sn': '0.0#', 'tstamp': '1691940879'}
{'Sm': '99.9#', 'tstamp': '1691940879'}
{'Sx': '0.0#', 'tstamp': '1691940879'}
{'Dn': '000#', 'tstamp': '1691940880'}
{'Dm': '000#', 'tstamp': '1691940880'}
{'Dx': '000#', 'tstamp': '1691940880'}
{'Sn': '0.0#', 'tstamp': '1691940880'}
{'Sm': '99.9#', 'tstamp': '1691940880'}
{'Sx': '0.0#', 'tstamp': '1691940880'}
{'Dn': '000#', 'tstamp': '1691940881'}
{'Dm': '000#', 'tstamp': '1691940881'}
{'Dx': '000#', 'tstamp': '1691940881'}
{'Sn': '0.0#', 'tstamp': '1691940881'}
{'Sm': '99.9#', 'tstamp': '1691940881'}
{'Sx': '0.0#', 'tstamp': '1691940881'}
{'Dn': '000#', 'tstamp': '1691940882'}
{'Dm': '000#', 'tstamp': '1691940882'}
{'Dx': '000#', 'tstamp': '1691940882'}
{'Sn': '0.0#', 'tstamp': '1691940882'}
{'Sm': '99.9#', 'tstamp': '1691940882'}
{'Sx': '0.0#', 'tstamp': '1691940882'}
{'Ta': '24.5C', 'tstamp': '1691940882'}
{'Ua': '44.2P', 'tst

KeyboardInterrupt: ignored